# **GPU and CPU Particle Swarm Optimization (PSO) Repository**

This repository implements a parallelized Particle Swarm Optimization (PSO) algorithm in both CPU and GPU versions, designed for optimizing a variety of benchmark objective functions. The repository demonstrates the speed advantages of GPU computation by leveraging **CuPy** for GPU-based calculations, as well as the traditional **NumPy** library for CPU-based calculations.

### **Key Features:**
1. **Benchmark Objective Functions**:
   - Implementations of popular optimization functions such as **Sphere**, **Rosenbrock**, **Rastrigin**, **Ackley**, **Schwefel**, **Griewank**, **Levy**, and **Michalewicz** for testing and optimizing.
   - Functions are optimized for both CPU and GPU environments, ensuring compatibility with a wide range of hardware setups.

2. **Particle Swarm Optimization (PSO)**:
   - **CPU Implementation**: A standard PSO implementation using **NumPy** for sequential computation, ideal for smaller optimization problems or environments without GPU access.
   - **GPU Implementation**: A fully vectorized PSO implementation using **CuPy**, which allows for massively parallel processing on compatible GPUs, significantly accelerating optimization for large search spaces or high-dimensional problems.

3. **Performance Comparison**:
   - The repository includes a benchmark comparing the CPU and GPU versions of PSO. Performance is evaluated based on runtime, demonstrating a **speedup of up to 30x** when using GPU acceleration.
   - Results for **optimal positions**, **fitness values**, and **execution times** are provided to showcase the efficiency gains of GPU-based optimization.

4. **Customizability**:
   - Supports custom bounds and parameter configurations, such as swarm size, inertia weight (ω), and acceleration coefficients (cognitive and social components).
   - Flexible API for integration into custom optimization workflows, with easy-to-use function wrappers for different objective functions.

5. **Scalability**:
   - The GPU implementation efficiently handles large swarms and high-dimensional problems, making it suitable for use cases in areas like machine learning, data science, engineering, and financial optimization.

### **Usage**:
- Simply modify the bounds and swarm size to suit the problem at hand.
- Choose between the CPU or GPU versions based on your hardware and performance requirements.
- Benchmark performance with the provided objective functions or add custom functions as needed.

### **Benefits**:
- **Accelerated Computation**: The GPU-based version provides substantial performance improvements, especially for large and complex optimization problems.
- **Ease of Use**: Simple function calls allow users to quickly integrate PSO into their optimization tasks, with minimal setup required.
- **Versatility**: Supports a wide range of problems from standard benchmark functions to custom real-world optimization tasks.

This repository is ideal for anyone looking to perform high-dimensional optimization using PSO, with the added benefit of GPU acceleration for faster results.

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
import time
import cupy as cp
from functools import partial


# Cost Functions


### Objective Functions for PSO Optimization

This repository includes several commonly used benchmark functions for evaluating optimization algorithms, including the **Particle Swarm Optimization (PSO)**, with both **GPU** and **CPU** implementations using **CuPy** (GPU) and **NumPy** (CPU). These functions are designed to test the performance of optimization algorithms in a variety of problem domains.

#### GPU-Based Objective Functions (CuPy)

1. **Sphere Function**:
   A simple test function used to evaluate the convergence of optimization algorithms. The function computes the sum of the squares of the input vector elements.
   ```python
   cp.sum(X**2, axis=1)
   ```

2. **Michalewicz Function**:
   A multimodal function that features several local minima, making it a challenging optimization problem.
   ```python
   -cp.sum(cp.sin(X) * cp.sin((X**2) / cp.pi)**(2 * m), axis=1)
   ```

3. **Schwefel Function**:
   A highly multimodal function known for its large global optima and challenging local minima.
   ```python
   418.9829 * X.shape[1] - cp.sum(X * cp.sin(cp.sqrt(cp.abs(X))), axis=1)
   ```

4. **Rosenbrock (Banana) Function**:
   This function is widely used in optimization literature due to its narrow, curved valley containing the global minimum.
   ```python
   cp.sum(100 * (X[:, 1:] - X[:, :-1]**2)**2 + (1 - X[:, :-1])**2, axis=1)
   ```

5. **Griewank Function**:
   A multimodal function often used in optimization. The objective is to minimize a combination of a quadratic sum and a product of cosine terms.
   ```python
   sum_term = cp.sum(X**2, axis=1) / 4000
   prod_term = cp.prod(cp.cos(X / cp.sqrt(cp.arange(1, X.shape[1] + 1))), axis=1)
   ```

6. **Rastrigin Function**:
   A well-known benchmark in optimization that exhibits a large number of local minima, making it challenging for algorithms to find the global minimum.
   ```python
   10 * n + cp.sum(X**2 - 10 * cp.cos(2 * cp.pi * X), axis=1)
   ```

7. **Ackley Function**:
   A complex, multimodal function used to test optimization algorithms. It features a nearly flat outer region and a narrow, deep hole at the global minimum.
   ```python
   -a * cp.exp(-b * cp.sqrt(sum1 / n)) - cp.exp(sum2 / n) + a + cp.exp(1)
   ```

8. **Levy Function**:
   A highly multimodal function used for testing optimization algorithms that requires searching through a complex landscape of peaks and valleys.
   ```python
   np.sin(np.pi * w[0])**2 + np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2)) + (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)
   ```

---

#### CPU-Based Objective Functions (NumPy)

1. **Sphere Function**:
   Same as the GPU-based Sphere function but using NumPy. The fitness is calculated as the sum of squared inputs.
   ```python
   np.sum(x**2)
   ```

2. **Rosenbrock Function**:
   Similar to the GPU-based version, this function evaluates the banana-shaped valley.
   ```python
   np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)
   ```

3. **Rastrigin Function**:
   The CPU version of the Rastrigin function.
   ```python
   10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))
   ```

4. **Ackley Function**:
   The CPU implementation of the Ackley function, which is complex and multimodal.
   ```python
   -20 * np.exp(-0.2 * np.sqrt(np.mean(x**2))) - np.exp(np.mean(np.cos(2 * np.pi * x))) + 20 + np.e
   ```

5. **Griewank Function**:
   The CPU version of the Griewank function.
   ```python
   1 + np.sum(x**2) / 4000 - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x)+1))))
   ```

6. **Schwefel Function**:
   The CPU implementation of the Schwefel function.
   ```python
   418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))
   ```

7. **Levy Function**:
   The CPU version of the Levy function.
   ```python
   np.sin(np.pi * w[0])**2 + np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2)) + (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)
   ```

8. **Michalewicz Function**:
   The CPU version of the Michalewicz function.
   ```python
   -np.sum(np.sin(x) * np.sin((np.arange(1, len(x)+1) * x**2) / np.pi)**(2*m))
   ```

---

These functions can be used to benchmark PSO (and other optimization algorithms) on a variety of optimization landscapes. Depending on the problem, the functions may require different strategies to find the global minimum, with the ability to switch between GPU and CPU implementations based on hardware availability.

In [ ]:
# === Sample Objective Function ===
def sphere_function_gpu(X):
    """Vectorized Sphere Function: Computes fitness for all particles in parallel"""
    return cp.sum(X**2, axis=1)  # Sum along each particle's dimension

def michalewicz_function_gpu(X, m=10):
    return -cp.sum(cp.sin(X) * cp.sin((X**2) / cp.pi)**(2 * m), axis=1)

def schwefel_function_gpu(X):
    return 418.9829 * X.shape[1] - cp.sum(X * cp.sin(cp.sqrt(cp.abs(X))), axis=1)

def rosenbrock_function_gpu(X):
    return cp.sum(100 * (X[:, 1:] - X[:, :-1]**2)**2 + (1 - X[:, :-1])**2, axis=1)

def griewank_function_gpu(X):
    sum_term = cp.sum(X**2, axis=1) / 4000
    prod_term = cp.prod(cp.cos(X / cp.sqrt(cp.arange(1, X.shape[1] + 1))), axis=1)
    return sum_term - prod_term + 1

def rastrigin_function_gpu(X):
    """Vectorized Rastrigin Function"""
    n = X.shape[1]
    return 10 * n + cp.sum(X**2 - 10 * cp.cos(2 * cp.pi * X), axis=1)

def ackley_function_gpu(X):
    """Vectorized Ackley Function"""
    a, b, c = 20, 0.2, 2 * cp.pi
    n = X.shape[1]
    sum1 = cp.sum(X**2, axis=1)
    sum2 = cp.sum(cp.cos(c * X), axis=1)
    term1 = -a * cp.exp(-b * cp.sqrt(sum1 / n))
    term2 = -cp.exp(sum2 / n)
    return term1 + term2 + a + cp.exp(1)

def levy_function_gpu(X):
    """Vectorized Levy Function"""
    w = 1 + (X - 1) / 4
    term1 = cp.sin(cp.pi * w[:, 0])**2
    term2 = cp.sum((w[:, :-1] - 1)**2 * (1 + 10 * cp.sin(cp.pi * w[:, :-1] + 1)**2), axis=1)
    term3 = (w[:, -1] - 1)**2 * (1 + cp.sin(2 * cp.pi * w[:, -1])**2)
    return term1 + term2 + term3



# 1. Sphere function (already provided)
def sphere_function_cpu(x):
    return np.sum(x**2)

# 2. Rosenbrock function (banana function)
def rosenbrock_function_cpu(x):
    return np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

# 3. Rastrigin function
def rastrigin_function_cpu(x):
    return 10 * len(x) + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

# 4. Ackley function
def ackley_function_cpu(x):
    return -20 * np.exp(-0.2 * np.sqrt(np.mean(x**2))) - \
           np.exp(np.mean(np.cos(2 * np.pi * x))) + 20 + np.e

# 5. Griewank function
def griewank_function_cpu(x):
    return 1 + np.sum(x**2) / 4000 - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x)+1))))

# 6. Schwefel function
def schwefel_function_cpu(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

# 7. Levy function
def levy_function_cpu(x):
    w = 1 + (x - 1) / 4
    return np.sin(np.pi * w[0])**2 + \
           np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2)) + \
           (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)

# 8. Michalewicz function
def michalewicz_function_cpu(x, m=10):
    return -np.sum(np.sin(x) * np.sin((np.arange(1, len(x)+1) * x**2) / np.pi)**(2*m))


# Numpy (CPU Version PSO)


### **Particle Swarm Optimization (PSO) - CPU Implementation**

This function implements the **Particle Swarm Optimization (PSO)** algorithm on the CPU using **NumPy**. PSO is an optimization technique inspired by the social behavior of birds flocking or fish schooling. It searches for the optimal solution by iteratively improving a population of candidate solutions (particles) based on both their own experience and the experience of their neighbors.

#### **Function Overview:**

```python
def pso_cpu(func, lb, ub, args=(), kwargs={},
            swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=100,
            particle_output=False):
```

- **`func`**: The objective function to be optimized. It takes an input vector `x` and returns a scalar value representing the fitness.
- **`lb`** and **`ub`**: Lower and upper bounds of the search space for each dimension of the particles' positions.
- **`args`** and **`kwargs`**: Additional arguments and keyword arguments to be passed to the objective function `func`.
- **`swarmsize`**: The number of particles in the swarm (default is 100).
- **`omega`**: The inertia weight, which controls how much the previous velocity influences the current velocity (default is 0.5).
- **`phip`**: The cognitive component, controlling how much the particle is attracted to its own best position (default is 0.5).
- **`phig`**: The social component, controlling how much the particle is attracted to the global best position (default is 0.5).
- **`maxiter`**: The maximum number of iterations for the optimization process (default is 100).
- **`particle_output`**: If `True`, returns the final positions and fitness values of all particles along with the best solution found.

#### **Algorithm Workflow:**

1. **Initialization**:
   - The positions (`x`) and velocities (`v`) of each particle are randomly initialized within the bounds `lb` and `ub`.
   - The fitness of each particle is evaluated using the objective function `func`.
   - Each particle’s personal best position (`p`) is initialized as its current position.
   - The global best position (`g`) is initialized as the best personal best position in the swarm.

2. **Optimization Loop** (for `maxiter` iterations):
   - **Velocity Update**: The velocity of each particle is updated using a combination of its previous velocity, its personal best position, and the global best position.
   - **Position Update**: Each particle's position is updated by adding its velocity, with clamping to ensure the positions stay within the bounds.
   - **Fitness Evaluation**: The fitness of the updated positions is computed.
   - **Personal Best Update**: If a particle's new position yields a better fitness, its personal best position is updated.
   - **Global Best Update**: The global best position is updated if any particle’s fitness improves the global best.

3. **Output**:
   - The function prints the progress of the optimization in each iteration, showing the best solution found so far and its fitness value.
   - After completing all iterations, the function returns the global best position and fitness. If `particle_output` is set to `True`, it also returns the final positions and fitness values of all particles.

#### **Key Parameters**:
- **Inertia (`omega`)**: Controls the influence of a particle's previous velocity on its current velocity. A lower value encourages exploration, while a higher value encourages exploitation.
- **Cognitive Component (`phip`)**: Controls the influence of the particle's own best position.
- **Social Component (`phig`)**: Controls the influence of the global best position.



In [2]:
def _obj_wrapper(func, args, kwargs, x):
    """Objective function wrapper to support function evaluation"""
    return func(x, *args, **kwargs)

def pso_cpu(func, lb, ub, args=(), kwargs={},
            swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=100,
            particle_output=False):
    """CPU implementation using NumPy"""

    # Convert bounds to NumPy arrays
    lb = np.asarray(lb, dtype=np.float32)
    ub = np.asarray(ub, dtype=np.float32)

    assert len(lb) == len(ub), 'Bounds must have same dimension'
    assert np.all(ub > lb), 'All upper bounds must > lower bounds'

    vhigh = np.abs(ub - lb)
    vlow = -vhigh

    # Initialize objective function wrapper
    obj = partial(_obj_wrapper, func, args, kwargs)

    # Initialize swarm
    S, D = swarmsize, len(lb)
    x = lb + (ub - lb) * np.random.rand(S, D)  # Positions
    v = vlow + (vhigh - vlow) * np.random.rand(S, D)  # Velocities

    p = x.copy()
    fx = np.array([obj(x[i]) for i in range(S)])  # Evaluate fitness
    fp = fx.copy()
    g = p[np.argmin(fp)].copy()
    fg = np.min(fp)

    # Optimization loop
    for it in range(maxiter):
        rp = np.random.rand(S, D)
        rg = np.random.rand(S, D)

        # Vectorized velocity & position update
        v = omega * v + phip * rp * (p - x) + phig * rg * (g - x)
        x = np.clip(x + v, lb, ub)  # Clamping within bounds

        # Evaluate all particles
        fx = np.array([obj(x[i]) for i in range(S)])

        # Update personal best (masking)
        improved = fx < fp
        p[improved] = x[improved]
        fp[improved] = fx[improved]

        # Update global best
        curr_min_idx = np.argmin(fp)
        if fp[curr_min_idx] < fg:
            fg = fp[curr_min_idx]
            g = p[curr_min_idx].copy()

        # Print iteration progress
        print(f'Iteration {it+1}: Best solution = {g}, Fitness = {fg}')

    print(f'Optimization completed after {maxiter} iterations')

    if particle_output:
        return g, fg, p, fp
    return g, fg


In [42]:
# Bounds
lb = [-10, -10, -10, -10, -10, -10, -10, -10,]
ub = [10, 10, 10, 10, 10, 10, 10, 10]

# Run PSO
best_pos, best_fitness = pso_cpu(michalewicz_function_cpu, lb, ub, swarmsize=10000, maxiter=100)

print("Optimal Position:", best_pos)
print("Optimal Fitness:", best_fitness)


Iteration 1: Best solution = [-0.71647389 -1.61143646 -3.54075019 -2.53220402 -1.29450045  2.66161129
 -0.7074832   0.63165293], Fitness = 31.718455395745025
Iteration 2: Best solution = [-0.71647389 -1.61143646 -3.54075019 -2.53220402 -1.29450045  2.66161129
 -0.7074832   0.63165293], Fitness = 31.718455395745025
Iteration 3: Best solution = [ 1.9222638  -0.47857874  2.25624799  1.15852028  0.45362385 -2.54006401
  0.39509325  0.35618643], Fitness = 17.297627206869933
Iteration 4: Best solution = [ 0.59881567 -0.6914498  -0.3002888  -0.77578553  0.26476432 -0.18048924
 -0.22901889  0.96522721], Fitness = 2.6154893113390427
Iteration 5: Best solution = [-0.52253949 -0.07270196 -0.21465605 -0.22335913  0.72155211  0.29246553
 -0.43862509 -0.07073265], Fitness = 1.1778682324367322
Iteration 6: Best solution = [ 0.37964009 -0.23677363  0.31969748 -0.12172622 -0.3021361   0.26819626
  0.12521387  0.12186335], Fitness = 0.5109567467215701
Iteration 7: Best solution = [ 0.08987014 -0.1471941

# Cupy (GPU Version PSO)


### **Particle Swarm Optimization (PSO) - GPU Implementation**

This function provides a **GPU-accelerated version** of the **Particle Swarm Optimization (PSO)** algorithm using **CuPy**. PSO is a population-based optimization algorithm that simulates the social behavior of birds or fish to find the optimal solution for a given objective function.

#### **Function Overview:**

```python
def pso_gpu(func, lb, ub, args=(), kwargs={},
            swarmsize=10000, omega=0.5, phip=0.5, phig=0.5, maxiter=100,
            particle_output=False):
```

- **`func`**: The objective function to be optimized. It accepts a position vector `x` and returns a scalar fitness value.
- **`lb`** and **`ub`**: Lower and upper bounds for each dimension of the particles' positions.
- **`args`** and **`kwargs`**: Additional arguments and keyword arguments to pass to the objective function `func`.
- **`swarmsize`**: Number of particles in the swarm (default is 10,000).
- **`omega`**: Inertia weight, influencing the particle's previous velocity (default is 0.5).
- **`phip`**: Cognitive coefficient, influencing the attraction of particles to their personal best (default is 0.5).
- **`phig`**: Social coefficient, influencing the attraction to the global best position (default is 0.5).
- **`maxiter`**: Maximum number of iterations (default is 100).
- **`particle_output`**: If `True`, returns the final positions and fitness of all particles in addition to the global best solution.

#### **Algorithm Workflow:**

1. **Initialization**:
   - The positions (`x`) and velocities (`v`) of all particles are initialized randomly within the bounds `lb` and `ub` using CuPy arrays to leverage GPU parallelism.
   - The fitness of all particles is evaluated simultaneously using the objective function `func`.
   - Each particle's personal best position (`p`) is initialized as its current position.
   - The global best position (`g`) is initialized as the position with the best fitness in the swarm.

2. **Optimization Loop** (for `maxiter` iterations):
   - **Random Numbers Pre-generation**: All random numbers for the iterations are generated in advance, stored in arrays, and used throughout the optimization to avoid repeated random number generation.
   - **Velocity and Position Update**: The velocity and position of each particle are updated in a fully vectorized manner, taking into account its previous velocity, its personal best, and the global best.
   - **Fitness Calculation**: The fitness of all particles is recalculated in parallel.
   - **Personal Best Update**: If any particle's fitness improves, its personal best position is updated.
   - **Global Best Update**: The global best position is updated if any particle’s fitness improves the current global best.

3. **Output**:
   - The function prints the best fitness found so far in each iteration.
   - After all iterations, the global best position and its fitness value are returned. If `particle_output` is `True`, the function also returns the final positions and fitness values of all particles.

4. **GPU to CPU Transfer**:
   - Before returning the results, the best solution is transferred back to the CPU using CuPy's `asnumpy()` function.

#### **Key Features**:
- **Fully Parallelized**: All particles’ positions, velocities, and fitness values are calculated in parallel using GPU acceleration, making this implementation much faster than the CPU version, especially for large swarm sizes.
- **Vectorized Operations**: The velocity and position updates, as well as the fitness evaluations, are fully vectorized to take advantage of GPU's parallel processing capabilities.
- **Pre-generated Random Numbers**: To improve efficiency, all random numbers for velocity updates and fitness evaluations are generated upfront for all iterations.
  

#### **Use Cases**:
- This GPU implementation of PSO is well-suited for large-scale optimization tasks, such as optimizing complex functions with many dimensions or using large particle swarms.
- It can be applied to problems in various domains, including engineering, machine learning (e.g., hyperparameter optimization), and finance.



In [4]:
def _obj_wrapper(func, args, kwargs, x):
    """Objective function wrapper to support function evaluation"""
    return func(x, *args, **kwargs)

def pso_gpu(func, lb, ub, args=(), kwargs={},
            swarmsize=10000, omega=0.5, phip=0.5, phig=0.5, maxiter=100,
            particle_output=False):
    """Fully parallelized GPU-based PSO using CuPy"""

    # Convert bounds to CuPy arrays
    lb = cp.asarray(lb, dtype=cp.float32)
    ub = cp.asarray(ub, dtype=cp.float32)

    assert len(lb) == len(ub), 'Bounds must have the same dimension'
    assert cp.all(ub > lb), 'All upper bounds must be greater than lower bounds'

    vhigh = cp.abs(ub - lb)
    vlow = -vhigh

    # Initialize objective function wrapper
    obj = partial(_obj_wrapper, func, args, kwargs)

    # Initialize swarm on GPU
    S, D = swarmsize, len(lb)
    x = lb + (ub - lb) * cp.random.rand(S, D)  # Positions
    v = vlow + (vhigh - vlow) * cp.random.rand(S, D)  # Velocities

    p = x.copy()

    # 🔥 **Vectorized Fitness Evaluation**
    fx = obj(x)  # Compute all particles at once
    fp = fx.copy()

    # Find global best
    min_idx = cp.argmin(fp)
    g = x[min_idx].copy()
    fg = fp[min_idx]

    # Pre-generate all random numbers on GPU
    all_rp = cp.random.rand(maxiter, S, D)
    all_rg = cp.random.rand(maxiter, S, D)

    # Optimization loop
    for it in range(maxiter):
        rp = all_rp[it]
        rg = all_rg[it]

        # 🔥 **Fully Vectorized Position & Velocity Updates**
        v = omega * v + phip * rp * (p - x) + phig * rg * (g - x)
        x = cp.clip(x + v, lb, ub)  # Clamping within bounds

        # 🔥 **Vectorized Fitness Calculation**
        fx = obj(x)

        # Update personal best
        improved = fx < fp
        p[improved] = x[improved]
        fp[improved] = fx[improved]

        # Update global best
        min_idx = cp.argmin(fp)
        if fp[min_idx] < fg:
            fg = fp[min_idx]
            g = p[min_idx].copy()

        # Print iteration progress
        print(f'Iteration {it+1}: Best Fitness = {fg}')

    print(f'Optimization completed after {maxiter} iterations')

    # Move results to CPU before returning
    g_cpu = cp.asnumpy(g)
    fg_cpu = float(fg)

    if particle_output:
        return g_cpu, fg_cpu, cp.asnumpy(p), cp.asnumpy(fp)
    return g_cpu, fg_cpu

# Example usage

The results of the comparison between the GPU-accelerated PSO and the CPU-based PSO are as follows:

### **CPU Results:**
- **Optimal Position**: `[-8.08e-16, 2.72e-16, -2.26e-15, -6.38e-15, -1.75e-14, 4.24e-15, 3.82e-15, -1.12e-14]`
- **Optimal Fitness**: `5.12e-28`
- **Runtime**: `5.80 seconds`

### **GPU Results:**
- **Optimal Position**: `[ 5.35e-15, 2.31e-14, -8.56e-15, -6.31e-15, -3.15e-15, -3.42e-15, 7.89e-15, -1.04e-14]`
- **Optimal Fitness**: `8.68e-28`
- **Runtime**: `0.19 seconds`

### **Speedup**: `29.82x`

#### **Observations**:
- The **GPU version** of PSO is significantly faster than the **CPU version**, with a **speedup of nearly 30x**.
- The **optimal fitness values** are very close, though the GPU version yielded a slightly higher fitness value. This difference is marginal, but it might be attributed to the stochastic nature of PSO.
- **GPU acceleration** is highly beneficial for large swarm sizes or problems with many dimensions, as seen in this example with 8 dimensions and 10,000 particles.

This demonstrates the efficiency of GPU-accelerated PSO for large optimization problems. The speedup can be even more pronounced for higher-dimensional problems or larger swarms.

In [47]:
# Bounds
lb = [-10] * 8
ub = [10] * 8

# Run PSO with CuPy acceleration
start_time_gpu = time.time()
best_pos_gpu, best_fitness_gpu = pso_gpu(sphere_function_gpu, lb, ub, swarmsize=10000, maxiter=100)
end_time_gpu = time.time()
gpu_runtime = end_time_gpu - start_time_gpu

print("\nGPU Results:")
print("Optimal Position:", best_pos_gpu)
print("Optimal Fitness:", best_fitness_gpu)
print(f"GPU Runtime: {gpu_runtime:.4f} seconds")

# Run PSO with CPU
start_time_cpu = time.time()
best_pos_cpu, best_fitness_cpu = pso_cpu(sphere_function_cpu, lb, ub, swarmsize=10000, maxiter=100)
end_time_cpu = time.time()
cpu_runtime = end_time_cpu - start_time_cpu

print("\nCPU Results:")
print("Optimal Position:", best_pos_cpu)
print("Optimal Fitness:", best_fitness_cpu)
print(f"CPU Runtime: {cpu_runtime:.4f} seconds")

# Compare runtimes
speedup = cpu_runtime / gpu_runtime
print(f"\nSpeedup: {speedup:.2f}x")


Iteration 1: Best Fitness = 31.372049549461593
Iteration 2: Best Fitness = 31.372049549461593
Iteration 3: Best Fitness = 14.8269499444637
Iteration 4: Best Fitness = 3.486220716039667
Iteration 5: Best Fitness = 1.5100013072035163
Iteration 6: Best Fitness = 0.8883963626929027
Iteration 7: Best Fitness = 0.40059198603028506
Iteration 8: Best Fitness = 0.182035029205679
Iteration 9: Best Fitness = 0.12164680395689584
Iteration 10: Best Fitness = 0.05002344709290669
Iteration 11: Best Fitness = 0.032849495683109874
Iteration 12: Best Fitness = 0.014621356856464977
Iteration 13: Best Fitness = 0.014621356856464977
Iteration 14: Best Fitness = 0.007258573638169048
Iteration 15: Best Fitness = 0.002646375642120031
Iteration 16: Best Fitness = 0.00249416674525907
Iteration 17: Best Fitness = 0.0007957542488888371
Iteration 18: Best Fitness = 0.0003591820991332555
Iteration 19: Best Fitness = 0.0001795945387366557
Iteration 20: Best Fitness = 0.00012267629943098638
Iteration 21: Best Fitness

In [45]:
# Bounds
lb = [-10] * 8
ub = [10] * 8

# Run PSO with CuPy acceleration
start_time_cpu = time.time()
best_pos_cpu, best_fitness_cpu = pso_cpu(rosenbrock_function_cpu, lb, ub, swarmsize=10000, maxiter=100)
end_time_cpu = time.time()
cpu_runtime = end_time_cpu - start_time_cpu

print("\nCPU Results:")
print("Optimal Position:", best_pos_cpu)
print("Optimal Fitness:", best_fitness_cpu)
print(f"CPU Runtime: {cpu_runtime:.4f} seconds")

Iteration 1: Best solution = [-1.22397451  1.27683727  1.78803627  2.38572472 -1.92949423 -1.61697864
 -0.12219697 -2.22253467], Fitness = 10006.823924913831
Iteration 2: Best solution = [-1.22397451  1.27683727  1.78803627  2.38572472 -1.92949423 -1.61697864
 -0.12219697 -2.22253467], Fitness = 10006.823924913831
Iteration 3: Best solution = [ 1.23610458  1.4459701  -0.74026846 -0.93437943  0.66480995 -0.05047985
  1.35904657 -2.08350482], Fitness = 2787.7870708893915
Iteration 4: Best solution = [ 0.05905013 -0.23808728  0.07054674  0.31764197 -0.39807186  0.36491748
 -0.85729617 -0.24844412], Fitness = 249.1545601109563
Iteration 5: Best solution = [ 0.97204575  1.07507571  0.29389587 -0.07691766 -0.12168079  0.71051645
  0.55592693  0.67266341], Fitness = 145.3629591194772
Iteration 6: Best solution = [ 0.36593447  0.00632497  0.07928638 -0.0056163  -0.07818543 -0.12612251
 -0.41466811  0.30607672], Fitness = 32.64914022744986
Iteration 7: Best solution = [ 0.88461429  0.65899576  

In [46]:
# Bounds
lb = [-10] * 8
ub = [10] * 8

# Run PSO with CuPy acceleration
start_time_gpu = time.time()
best_pos_gpu, best_fitness_gpu = pso_gpu(rosenbrock_function_gpu, lb, ub, swarmsize=10000, maxiter=100)
end_time_gpu = time.time()
gpu_runtime = end_time_gpu - start_time_gpu

print("\nGPU Results:")
print("Optimal Position:", best_pos_gpu)
print("Optimal Fitness:", best_fitness_gpu)
print(f"GPU Runtime: {gpu_runtime:.4f} seconds")

Iteration 1: Best Fitness = 10468.701874812921
Iteration 2: Best Fitness = 10468.701874812921
Iteration 3: Best Fitness = 340.1130414223711
Iteration 4: Best Fitness = 340.1130414223711
Iteration 5: Best Fitness = 123.22658140246187
Iteration 6: Best Fitness = 101.46330477181718
Iteration 7: Best Fitness = 26.77892400774286
Iteration 8: Best Fitness = 26.77892400774286
Iteration 9: Best Fitness = 26.77892400774286
Iteration 10: Best Fitness = 10.899009010460526
Iteration 11: Best Fitness = 10.899009010460526
Iteration 12: Best Fitness = 6.5616136849643585
Iteration 13: Best Fitness = 6.232641520751865
Iteration 14: Best Fitness = 5.9488011301812325
Iteration 15: Best Fitness = 5.406627602694227
Iteration 16: Best Fitness = 5.3978390615251355
Iteration 17: Best Fitness = 4.503997645569326
Iteration 18: Best Fitness = 4.503997645569326
Iteration 19: Best Fitness = 4.221812172330697
Iteration 20: Best Fitness = 4.037864139209122
Iteration 21: Best Fitness = 3.854152120611908
Iteration 22: